In [32]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [24]:
def read_dataframe(file_path):
    df = pd.read_parquet(file_path)

    # Calculating the duration in seconds
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td : td.total_seconds() / 60)

    # Taking duration Morethan 1 minute and Lessthan 1 hour
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    # Categorical and Numerical Variables to use
    categorical = ['PULocationID', 'DOLocationID']

    # One hot encoing Categorical Variables using Dictionary Vectorizer
    df[categorical] = df[categorical].astype(str)
    
    return df

In [25]:
# Loading dataframes
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [26]:
# Creating new columns
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [27]:
# List of columns to be used
categorical = ['PU_DO']
numerical = ['trip_distance']

# Initializing DictVectorizer
dv = DictVectorizer()

# Collecting train dicts
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

# Collecting Val dicts
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [28]:
# Collecting train and val y data
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [29]:
# Building Model
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

np.sqrt(mean_squared_error(y_val, y_pred))

7.479598548891981

In [33]:
# Saving the model
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)